# Funciones de un solo sentido

## Ejercicio 1

Sea $(a_1,\ldots,a_k)$ una secuencia super-creciente de números positivos (la suma de los términos que preceden a $a_i$ es menor que $a_i$, para todo $i$). Elige $n > \sum a_i$, y $u$ un entero positivo tal que $gcd(n, u) = 1$. Define $a_i^* = ua_i \bmod n$. La función mochila (_knapsack_) asociada a $(a_1^*,\ldots,a_k^*)$ es $$f:\mathbb{Z}_2^k \rightarrow \mathbb{N}, f(x_1,\ldots,x_k) = \sum_{i=1}^k x_ia_i^*$$

Implementa esta función y su inversa, tal y como se explica en _P. J. Cameron, Notes on cryptography_. La llave pública es $((a_1,\ldots,a_k), n, u)$.

In [1]:
import numpy as np
import AritmeticaModular as am
from random import randint

"""
    Generate a super-increasing sequence of length k.
    
    A super-increasing sequence has k elements where
    
                    a_i > sum(a_0,...,a_(i - 1))
                    
    An example: (1, 2, 4, 8, 16)
    
    k: length of the message's block
"""
def generate_super_increasing(k):
    
    # generate the first element of the sequence
    sequence = [0]*k
    sequence[0] = randint(1, k)
    
    for i in range(1, k):
        sequence[i] = sum(sequence[:i]) + randint(sequence[i-1], sequence[i-1]*3)
            
    return np.array(sequence, dtype = np.uint32)

generate_super_increasing(8)

array([    3,     6,    20,    76,   293,   882,  3242, 11488], dtype=uint32)

Con la función *generate_super_increasing* generamos una secuencia súper-creciente de longitud $k$, donde $k$ es el tamaño de bloque a cifrar del mensaje original.

A continuación, tenemos que tomar un número $n$ tal que $n > \sum_{i=0}^k a_i$ y un número $u$ que sea primo relativo con $n$, es decir, que $gcd(n, u) = 1$.

In [2]:
"""
    Generate the keys for the knapsack cipher.
    
    Parameters:
        block_length: length of the message's block
    Return:
        public_key: modified sequence
        private_key: original super-increasing sequence, n and u
"""

def generate_keys(block_length):
    # generate a sequence
    sequence = generate_super_increasing(block_length)
    
    n = sum(sequence) + sequence[randint(0,block_length)]*2
    
    found_u = False
    
    while not found_u:
        
        u = randint(1, n)
        
        if am.ext_euclides(n,u)[0] == 1: 
            found_u = True
            
    public_sequence = np.array(list(map(lambda a_i: (u*a_i) % n, sequence)), dtype = np.uint32)
    
    return public_sequence, (sequence, n, u)

key_pub, key = generate_keys(8)
print("Public key:", key_pub)
print("Private key:", key[0], key[1], key[2])

Public key: [41299  3700 18057 22591 42707  4563 26199 48812]
Private key: [    7    28    93   331  1007  3279 12987 36044] 53832 36661


Una vez generadas nuestras llaves, podemos pasar a realizar el cifrado del mensaje. Para cifrar el mensaje, al igual que hacíamos en el cifrado en flujo usando el generador de Geffe, tendremos que pasar el string a binario, y para ello, usaremos las funciones *str_to_binlist* y *bin_to_str*. Una vez realizado esto, podremos pasar a a cifrar el mensaje en bloques de tamaño $k$.

In [3]:
from CifradoFlujo import bin_to_str, str_to_binlist

"""
                            Knapsack cypher
    
    This function take a message and the public
    key, and encrypt the message performing:
            ___n
            \                 a_i * e_i
            /__ 0
            
    where a_i are the bits of the message and e_i
    the elements of the public key.
    
    Parameters:
        - message: a str containing the plain message.
        - key_pub: public key
"""
def knapsack_cypher(message, key_pub):
    # take the block size
    block_size = len(key_pub)
    # transform the message to binary list
    binary_message = str_to_binlist(message)
    # check if the message is divisible by the block size
    # if not, add zeros at the end
    while len(binary_message) % block_size != 0:
        binary_message.append(0)
    
    cypher, binary_message = [], np.array(binary_message)
     # cypher text
    for i in range(0,len(binary_message), block_size):
        cypher.append(np.sum(key_pub[binary_message[i:i + block_size] == 1]))
        
    return cypher


encrypted = knapsack_cypher("Lorem ipsum dolor sit amet", key_pub)

print("Encrypted text:", encrypted)

Encrypted text: [50970, 144038, 70547, 75132, 117839, 18057, 113276, 44348, 119359, 97723, 117839, 18057, 26320, 144038, 69027, 144038, 70547, 18057, 119359, 113276, 48911, 18057, 70569, 117839, 75132, 48911]


In [4]:
from AritmeticaModular import inverse
"""
                                        Knapsack decypher
                        
In order to decrypt a message encrypted with this algorithm,
it's necessary to calculate v as u^{-1} mod n using Euclids' 
algorithm.

Then, the message is decrypted:
        b 
"""
def knapsack_decypher(message, key):
    # unfold the private key and calculate 
    # the inverse of u mod n
    p_key, n, u = key
    v =  inverse(u, n)
    # Concatenate list in python it's very simple if you use the operator + and two list. 
    # That is the reason to make decypher_message an empty list
    decypher_message = []
    # start decypher
    for b in message:
        # b_ = vb mod n
        b_, aux = (b * v) % n, 0
        # start the greedy algorithm
        greedy_result = set()
        for i in range(1, len(p_key) + 1):
            if aux + p_key[-i] <= b_:
                aux += p_key[-i]
                greedy_result.add(p_key[-i])
                if aux == b_:
                    break
        # restore and add the message
        decypher_message += list(map(lambda x: 1 if x in greedy_result else 0, p_key))
        # return the plain text
    return bin_to_str(decypher_message)


knapsack_decypher(encrypted, key)

'Lorem ipsum dolor sit amet'

En la función de descifrado haremos uso de las propiedades de las listas en Python y el uso de _set_ para poder buscar los elementos con una mayor velocidad. Al realizar el algoritmo Greedy, iremos insertando los elementos en un _set_, ya que, una vez haya encontrado el algoritmo Greedy la solución, pasaremos a recomponer el bloque de mensaje. Para ello, usaremos una función _map_ que crea una lista de forma que si $x$ se encuentra en el resultado del Greedy, devuelve un 1 y en caso contrario devuelve un 0.

Esto nos devolverá una lista de unos y ceros que podemos concatenar entre sí usando el operador $+$ de las listas. Cuando tengamos el mensaje completo lo reconstruimos a texto plano usando la función *bin_to_str*.

## Ejercicio 2

Sea $p$ un (pseudo-)primo mayor o igual que vuestro número de identidad. Encuentra un elemento primitivo $\alpha$, de $\mathbb{Z}_p^*$ (se puede usar el libro "_Handbook of Applied Cryptography [2.132 (iv)]_"); para facilitar el criterio, es bueno escoger $p$ de forma que $\frac{p - 1}{2}$ sea también primo, y para ell usamos Miller-Rabin). Definimos $$f:\mathbb{Z}_p \rightarrow \mathbb{Z}_p, x\mapsto\alpha^x$$
Calcula el inverso de tu fecha de nacimiento con el formato AAAAMMDD.

__En lo que sigue, $p$ y $q$ son enteros primos, y $n = pq$.__

In [5]:
from os import environ
import AritmeticaModular as am
from random import randint
# initial number
p, β = int(environ["ID"]), int(environ["BIRTH"])
α = randint(2, p)
# search a prime p that (p-1) // 2 is prime too (probably)
while not (am.miller_rabin_test(p) and am.miller_rabin_test((p -1)//2)):
        p +=1

## Ejercicio 3

Sea $f:\mathbb{Z}_n \rightarrow \mathbb{Z}_n$ la función de Rabin: $f(x) = x^2$. Sea $n = 48478872564493742276963$. Sabemos que $f(12) = 144= f(37659670402359614687722)$. Usando esta información, calcula $p$ y $q$ (mirar la demostración de "_Lecture Notes on Cryptography_", Lemma 2.43.

## Ejercicio 4

Elige $a_0$ y $a_1$ dos cuadrados arbitrarios módulo $n$ ($n$ como en el Ejercicio 3). Sea $$h:\mathbb{Z}_2 \times (\mathbb{Z}_n)^*, h(b,x) = x^2a_0^ba_1^{1 - b}$$ Usa la función de Merkle-Damg\.{a}rd para implentar una función resumen tomando $h$ como función de compresión (esta $h$ fue definida por Glodwasser, Micali y Rivest). Los parámetros $a_0, a_1$ y $n$ se hacen públicos (la función debería admitir un parámtero en el que venga especificado el vector inicial).

## Ejercicio 5

Sea $p$ el menor primo entero mayor o igual que tu número de identidad, y sea $q$ el primer primo mayor o igual que tu fecha de nacimiento (AAAAMMDD). Selecciona $e$ tal que $gcd(e, (p -1)(q -1)) = 1$. Define la función RSA $$f:\mathbb{Z}_n \rightarrow \mathbb{Z}_n, x \mapsto x^e$$

Calcula el inverso de 123456790.